<a href="https://colab.research.google.com/github/mebalint/hgt_football_project/blob/main/hgt_m_csapat_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---

Mielőtt belevágnánk fontos megjegyzés, hogy ez a csapat nyerte a félév elején a sport szeletet, melynek átadására még nem került sor.

---





### Bevezetés

Ez a Google Collab a Haladó Gépi Tanulás tárgy az M csapatának notebookja. Az M csapat tagjai: Matúz Máté, Mészáros Bálint, Medgyes Csaba és Oroszki Norbert. A terv kezdetleges, a rendelkezésre álló adatok, illetve Benchmark-ok függvényében változhat. Alapjáraton Sentiment Analysis-t szeretnénk vizsgálni  IMDb értékelések alapján.

Az eredeti terv:

- 3 hét (4-6. hét): Irodalom és a rendelkezésre álló adatok értelmezése, Benchmark-ok átnézése.
- 1 hét (7. hét): Adatértelmezés, adattisztítás és feldogozás. 
-Esetleges újratervezés, hogyha a Sentiment Analysis nem valósítható meg.
-4 hét (8-11. hét): Modellezés.
-2 hét (12-13. hét): Eredmények összegzése, modell kiértékelése és tesztelése, dokumentáció elkészítése. 

Mivel a Sentiment Analysis kifejezetten jól sikerült, nem látjuk értelmét, hogy ezt a modellt fejlesszük, így a félév végéig egy új témát szeretnénk kidolgozni, labdarúgás eredményeit szeretnénk prediktálni. 

Az új projekt Notebook-ját egy másik Colab-ban található meg.

# Használt csomagok:

Először is a későbbi Python frissítések miatt a használt csomagok és függvények importálásával kezdjük. 

In [ ]:
import numpy as np
import pandas as pd
import os
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
import string
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, GlobalAvgPool1D

Most kössük össze a Google Drive-ot ezzel a Notebook-al, hogy közösen tudjunk dolgozni. Egy pop-up fel fog ugrani.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Eredmények:

A Sentiment Analysis eredménye:
- Loss: $0.1116$
- Accuracy: $0.9620$ 
- Val_loss: $0.4210$
- Val_accuracy: $0.8761$

Ebbe már nem szereténk több időt befektetni, ezzel az eredménnyel teljes mértékben kiegyezünk.

# IMDb adathalmazon Sentiment Analysis:

Az adat az alábbi (nyilvános) linken megtekinthető: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/metadata?fbclid=IwAR1jviGz_kIpD87PoL23WxaPrtNW8B84E0Zuf3aw6DnIvcaw-rcWSoXUWgU Először az en_core_web_sm modellt vizsgáljuk.

In [ ]:
path_current = '/content/drive/MyDrive/HGT M csapat'

os.chdir(path_current)

df = pd.read_csv('IMDB Dataset.csv')

nlp = spacy.load('en_core_web_sm') 
 
#en_core_web_sm, this is the small english pipeline optimized for CPU. Small language model in short. 

Címkézzük az adatokat, 1-et írjunk pozitív esetben (dokumentáció: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

In [ ]:
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment']) 

# transformed column in space, output 0-s,1-s
# LabelEncoder fitted itself, found the unique values, and encoded them from 0,1,2,3.... now only 0,1.

Mivel értelmes dolgot szeretnénk visszakapni így az attribútumok után kell _ karakter.

In [ ]:
list(le.classes_)

['negative', 'positive']

Transzformáljuk és formázzuk a sztringeket, szabaduljunk meg a HTML tag-ektől stb. Példának nézzük az elsőt:

In [ ]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [ ]:
nlp.analyze_pipes() # We can see here the pipes our doc flows through

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False}},
 'problems': {'tok2vec': [],
  'tagger': [],
  'parser': [],
  'attribute_ruler': [],
  'lemmatizer': [],
  'ner': []},
 'att

Mivel ez csúnya transzformájunk:

In [ ]:
df['review'] = df['review'].apply(lambda x:x.replace('<br /><br />', ''))
df['review'] = df['review'].apply(lambda x:x.replace('..', ''))
df['review'] = df['review'].apply(lambda x:x.replace('...', ''))
df['review'] = df['review'].apply(lambda x:x.replace('....', ''))

Most csak a tokenizáló lépést csináljuk, Punctuation metódussal. Ez minden ASCII karaktert kigyűjt. (https://docs.python.org/3/library/string.html)

In [ ]:
punct = string.punctuation # List of all punctuation characters. 

def tokenizer_f(x): # We give it a dataframe. Or a row of a dataframe. 
    sent = nlp(x) # Creates a spacy doc object by running it through the pipeline
    token = []
    for i in (sent): # We iterate through the third biggest container in spacy the tokens.
        if not i.is_stop and not i.lemma_.lower() in punct: # Ha nincs stopwords-ben és nem punctutation.
            token.append(i.lemma_.lower())          
    return ' '.join(token) # megszépítve összerakjuk újra. 

# if i.lemma_ == '-PRON-': # Spacy_v2-ben ez még '-PRON-' volt. 
#    token.append(i.lower_)

In [ ]:
review = []
for i in tqdm(range(df.shape[0])):
    review.append(tokenizer_f(df['review'][i]))

# %store review Ezt érdemes lementeni 31 percig töltött. 

100%|██████████| 50000/50000 [38:18<00:00, 21.75it/s]


In [ ]:
X = np.array(review)
y = df['sentiment'].values

In [ ]:
np.shape(X)

(50000,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify = y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((33500,), (16500,), (33500,), (16500,))

In [ ]:
tokenizer = Tokenizer(num_words=10000,oov_token="<00v>")

tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen=400)
X_test = pad_sequences(X_test, maxlen=400)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = 10000, output_dim = 15,input_length = 400))
model.add(GlobalAvgPool1D())
model.add(Dense(15, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=2)

Epoch 1/10
1047/1047 - 8s - loss: 0.4705 - accuracy: 0.7713 - val_loss: 0.2935 - val_accuracy: 0.8868 - 8s/epoch - 7ms/step
Epoch 2/10
1047/1047 - 6s - loss: 0.2484 - accuracy: 0.9048 - val_loss: 0.2799 - val_accuracy: 0.8906 - 6s/epoch - 6ms/step
Epoch 3/10
1047/1047 - 7s - loss: 0.2090 - accuracy: 0.9210 - val_loss: 0.2883 - val_accuracy: 0.8876 - 7s/epoch - 6ms/step
Epoch 4/10
1047/1047 - 6s - loss: 0.1844 - accuracy: 0.9321 - val_loss: 0.2982 - val_accuracy: 0.8885 - 6s/epoch - 6ms/step
Epoch 5/10
1047/1047 - 6s - loss: 0.1678 - accuracy: 0.9380 - val_loss: 0.3084 - val_accuracy: 0.8886 - 6s/epoch - 6ms/step
Epoch 6/10
1047/1047 - 6s - loss: 0.1543 - accuracy: 0.9429 - val_loss: 0.3284 - val_accuracy: 0.8847 - 6s/epoch - 6ms/step
Epoch 7/10
1047/1047 - 6s - loss: 0.1415 - accuracy: 0.9491 - val_loss: 0.3681 - val_accuracy: 0.8745 - 6s/epoch - 6ms/step
Epoch 8/10
1047/1047 - 6s - loss: 0.1312 - accuracy: 0.9535 - val_loss: 0.3626 - val_accuracy: 0.8815 - 6s/epoch - 6ms/step
Epoch 9/